In [1]:

from transformers import AutoTokenizer, AutoModel
model_path = "/kaggle/input/chatglm3-6b/pytorch/6b/6"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True, device='cuda')
model = model.eval()


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [33]:
text = "你好，我是人工智能助手。"
print(f"1.用户的提问：{text}")

seg_words = tokenizer.tokenize(text)
print(f"2.用户的提问 分词结果：{seg_words}")

seg_word_ids = tokenizer.convert_tokens_to_ids(seg_words)
print(f"3.用户的提问 分词编码结果：{seg_word_ids}")


model_inputs = tokenizer([text], return_tensors="pt").to("cuda")
print(f"4.直接将 用户提问转为分词编码结果：{model_inputs}")

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
print(f"5.直接将用户提问转为分词编码结果：{generated_ids}")


generated_seg_word = tokenizer.convert_ids_to_tokens(generated_ids[0])
print(f"6.将模型的回复反编码为token：{generated_seg_word}")


response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(f"7.将模型的回复反编码为token,直接将模型回复反编码并合并为句子结果：{response}")

1.用户的提问：你好，我是人工智能助手。
2.用户的提问 分词结果：['▁你', '好', '，', '我是', '人工智能', '助手', '。']
3.用户的提问 分词编码结果：[36474, 54591, 31123, 33030, 34797, 42481, 31155]
4.直接将 用户提问转为分词编码结果：{'input_ids': tensor([[64790, 64792, 36474, 54591, 31123, 33030, 34797, 42481, 31155]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'position_ids': tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8]], device='cuda:0')}
5.直接将用户提问转为分词编码结果：tensor([[64790, 64792, 36474, 54591, 31123, 33030, 34797, 42481, 31155, 48895,
         38549, 31645, 31404, 42693, 33277, 31639, 40648, 55268, 55353, 36295,
         31514,     2]], device='cuda:0')
6.将模型的回复反编码为token：['[gMASK]', 'sop', '▁你', '好', '，', '我是', '人工智能', '助手', '。', '很高兴', '为您', '服务', '！', '请问', '有什么', '问题', '我可以', '帮', '您', '解答', '？', '']
7.将模型的回复反编码为token,直接将模型回复反编码并合并为句子结果：['[gMASK] sop 你好，我是人工智能助手。很高兴为您服务！请问有什么问题我可以帮您解答？']


In [34]:

from sentencepiece import SentencePieceProcessor
model_path = "/kaggle/input/chatglm3-6b/pytorch/6b/6/tokenizer.model"
sp_model = SentencePieceProcessor(model_file=model_path)
text = "你好，你是谁？"

tokens = sp_model.EncodeAsPieces(text)
print(f"1.句子转为tokens结果： {tokens}")

ids = sp_model.EncodeAsIds(text)
print(f"2.句子转为ids结果： {ids}")
decode_text = sp_model.Decode(ids)
print(f"3.ids转为句子结果：{decode_text}")

1.句子转为tokens结果： ['▁你', '好', '，', '你是', '谁', '？']
2.句子转为ids结果： [36474, 54591, 31123, 34607, 55622, 31514]
3.ids转为句子结果：你好，你是谁？
